## 2. Importing the libraries

In [ ]:
import numpy as np
import pandas as pd
import re
import spacy
import string
import tensorflow as tf
import nltk
from nltk.corpus import stopwords
pd.options.mode.chained_assignment = None
import re
import warnings
# Disable warnings
warnings.filterwarnings('ignore')
import gensim
import nltk
import pandas as pd
import spacy
from collections import Counter
from nltk.corpus import stopwords
from nltk import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from wordcloud import WordCloud
import tensorflow as tf
from nltk.stem.porter import PorterStemmer
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
import numpy as np
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from tensorflow.keras.optimizers import Adam
from transformers import TFTrainingArguments , TFBertForSequenceClassification , TFTrainer
import tensorflow as tf
import pickle

nltk.download('punkt')
nltk.download('stopwords')

## 3. Loading the data

In [ ]:
df = pd.read_csv("../data/preprocessed_df.csv")
df.head()

In [ ]:
lbl_encoder = LabelEncoder()
df['subreddit'] = lbl_encoder.fit_transform(df['subreddit'])

In [ ]:


with open('lbl_encoder.pkl', 'wb') as file:
    pickle.dump(lbl_encoder, file)

In [ ]:
X = tf.data.Dataset.from_tensor_slices((df['Sentence'].values))
y = tf.data.Dataset.from_tensor_slices((df['subreddit'].values))

In [ ]:
TEST_SIZE = .2

X_train ,X_test,y_train,y_test = train_test_split(df['Sentence'].values, df['subreddit'].values , test_size =TEST_SIZE )

In [ ]:



# Load the tokenizer and tokenize the data
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
TX_train = tokenizer(X_train.tolist(), return_tensors="tf", padding=True ,truncation=True)
TX_test = tokenizer(X_test.tolist() , return_tensors="tf", padding=True ,truncation=True)



In [ ]:
test_dataset = tf.data.Dataset.from_tensor_slices((
dict(TX_test) , y_test))

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((
dict(TX_train) , y_train))

In [ ]:

train_args = TFTrainingArguments(
    output_dir='./results',          # output directory
    overwrite_output_dir = True,
    num_train_epochs=2,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    eval_steps=100,
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    save_total_limit=8

)

In [ ]:
with train_args.strategy.scope():
        model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=6)

trainer = TFTrainer(
    model=model,
    args=train_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,


)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# save tokenizer

tokenizer.save_pretrained('models/tokenizer.h5')

# save model

model.save_pretrained('models/model.h5')



# Save model to HuggingFace Hub

In [ ]:
!transformers-cli login
# upload fine-tuned model to huggingface.co
!transformers-cli upload models/model.h5
# upload the tokenizer
!transformers-cli upload models/tokenizer.h5

* Created by : Wassim EL BAKKOURI , Fatima Zahra MOUMENE